# Welcome to the Bokeh Lab! 
## Intro
By now, you have seen and used some Python libraries and modules such as tutle, Pygame, NumPy, Matplotlib, and the datascience library developed here at Berkeley. This lab is somewhat of an extension of the Data Science lab. In recent years, Python has become a popular language for Data Science due to its versitility, ease to use, and robust data-science-related libraries.In this lab, we will introduce you to another Python library -- Bokeh, a library for interactive data visualization. We chose Bokeh because:
1. Being able to effectively communicate your findings with other people through data visualization is one of the most important aspects of Data Science, and you can use Bokeh to produce beautiful, interactive graphics!
2. Bokeh is as powerful as other popular data visualization libraries like Matplotlib and is less complex to learn.
3. Bokeh renders its graphics with HTML and JavaScript, making building web-based dashboards easy. For this lab, we will be outputting our graphs in this notebook, but you can easily output your work as an HTML file.

In this lab, we will walk you through two examples of visualizing COVID-19 data with Bokeh. The first exercise will be plotting the number of confirmed COVID cases in California from Spet.1 to Oct.30, and the second exercise will focus on creating a map of the U.S. based each state's confirmed number of COVID-19 cases on Oct.28.

Before we get started, we would like to mention that as you saw in lab 17, data analysis consists of multiple parts, so Bokeh often needs to be used with other libraries, such as pandas and NumPy, to produce the final result. For the purpose of this lab, we have abstracted that away for you. But if you are interested, we definitely encourage you to check them out; there are many resources available online to help you learn how to use them. We also encourage you to check out the [Bokeh documentation](https://docs.bokeh.org/en/latest/docs/user_guide.html) as well as their [gallery](https://docs.bokeh.org/en/latest/docs/gallery.html) to see what other cool things people have done with Bokeh. And if you are interested in Data Science, check out classes like Data 8 and Data C100!

## Data Sets
For this lab, we have provided you with two data sets. For more information on where the data come from, please refer to the "Sources/Acknowledgments" section below. If you have any questions/comment/concerns on how the two data sets are made, please feel free to reach out to us.

### 1. "covid_cali.csv": number of confirmed COVID-19 cases in California from Sept. 1 to Oct. 30: 
- `Month`: month when the data are gathered.



- `Date`: date when the data are gathered.



- `Confirmed`: Aggregated case count for the state.


- `People Tested`: Total number of people who have been tested for the state.

### 2. "covid_map.shp": COVID data for each state and geo data that help construct the shape of each state:
- `NAME`: name of each state; each state represent a row of the table. There are 52 rows since the data set includes Washington D.C. and Puerto Rico.


- `Confirmed`: Aggregated case count for the state.


- `Deaths`:  Aggregated death toll for the state.


- `Recovered`: Aggregated Recovered case count for the state.


- `Active`: Aggregated unresolved confirmed cases. Calculated by subtracting total recovered and total deaths from total cases.


- `People_Tes`: Total number of people who have been tested for the state.


- `geometry`: Coordinates that outline the shape of the state. 

## Sources/Acknowledgements:
- Data for the "covid_cali.csv" file are pulled from the COVID-19 U.S. daily reports from Sept.1 to Oct. 30 published by the COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University. [Here is the link to the repository](https://github.com/CSSEGISandData/COVID-19).
- COVID data for "covid_map.shp" are pulled from the COVID-19 U.S.daily reports on Oct. 28 published by the COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University. 
- Geodata in "covid_map.shp" used to construct the map of the U.S. come from the U.S. Census Bureau. Specifically, we used the file "cb_2018_us_state_20m.shp" to get the shape of each state. [Here is the link to the data set](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html)
- The second part of the lab is inspired by the [New York Times COVID-19 map](https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html), and during our implementation, we referred to the work of Rebecca Wang, as published in her [Medium article](https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9).
- We referred to the [Bokeh Documentation](https://docs.bokeh.org/en/latest/docs/user_guide.html) when we got stuck while developing this lab, and if you were ever confused on any part of the lab, we strongly encourage you to take a look at the documentation as well.

# Part 0: Getting Started

First, let's import all the libraries and submodules we need for this lab. Pandas, NumPy, and GeoPandas are libraries we need to read files and extract data. The rest are submodules of Bokeh we need to make our graphs. The final line makes sure that the graphs will show up in our notebook.

**Make sure you load the following cell before continuing with the rest of the lab!**

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.models import (Legend,
                          ColumnDataSource,
                          LinearColorMapper, 
                          BasicTicker,
                          BasicTickFormatter,
                          PrintfTickFormatter,
                          ColorBar,
                          FactorRange,
                          LogColorMapper,
                          GeoJSONDataSource,
                          HoverTool)
from bokeh.transform import dodge
from bokeh.palettes import *
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

After running the cell above, if you see "BokehJS 2.1.1 successfully loaded," congratulations, you have imported all the dependencies and are good to go!

# Part 1: Line Plot with Bokeh

First, let's examine the structure and the first 5 rows of our first table, "covid_cali," which contains the number of confirmed COVID-19 cases in the months of September and October in California. If you would like to see more of the table, feel free to put a number in `.head()`.

In [4]:
covid_cali = pd.read_csv('covid_cali.csv')
covid_cali.head()

Month  Date  Confirmed  People Tested
0  Sept     1     715617     11470696.0
1  Sept     2     721281     11580370.0
2  Sept     3     726018     11663924.0
3  Sept     4     730662     11796970.0
4  Sept     5     735314     11916323.0

Now, let's figure out a way to visually represent our data. One way we could do this is by makeing a bar graph of how many confirmed cases there are every day. From our observation of the table, we notice that we can first split the data into 2 categories: data from the month of September and data from October. Then, each category, we should have 30 bars that represent the daily confirmed cases count for that month.

To do this, we need a variable called `factors`. `factors` is list of **tupules**. Each tupule should have two items, the month and the date as **strings**. `factors` should look something like this: `[('Sept', '1'),('Sept','2')...]`. Fill out what factors should be in the first line. You can add a `print()` statement in the cell below to see if you have did this correctly.

**Hint 1**: try using list comprehension to turn entries of the `Date` column into **strings** first. You can use Python's built-in function `str()`.


**Hint 2**: Python's built-in `zip` function takes two lists and "zips" the item at index i of the first list and item at index i of the second list together into a tupule. For example, `list(zip([1,2,3],['a','b','c']))` would give you `[(1,'a'),(2,'b'),(3,'c')]`

**Hint 3** :You can get the data in a given column from the `covid_cali` table as a list with this syntax: `covid_cali[*column_name_here*]`

In [5]:
factors = #TODO:YOUR CODE HERE
print(factors)

SyntaxError: invalid syntax (<ipython-input-5-4959dca33ecf>, line 1)

After we have define `factors`, let's specify how tall our bars should be. The height of each bar should be the number of confirmed cases for a given day. Fill out what `num` should be. Remember that you could access the data from a specific column with `covid_cali[*column_name_here*]`!

In [1]:
graph = figure(x_range=FactorRange(*factors), plot_height=500,
           toolbar_location=None, tools="")

num = #TODO:YOUR CODE HERE
graph.vbar(x=factors, top=num, width=0.5, alpha=0.5)

#stylistic choices, feel free to read the Bokeh documentation and customize how your graph looks like!
graph.y_range.start = 0
graph.xaxis.major_label_orientation = 0.5
graph.xaxis.major_label_text_font_size = '0.2cm'
graph.xgrid.grid_line_color = None

show(graph)

SyntaxError: invalid syntax (<ipython-input-1-1f0c9f431596>, line 4)

We can see an upward trend from the graph above, However, it is hard to see the exact shape of the line, so run the cell below to see another way of visualizing our data. 

**Question: What other ways can you think of to visualize our data? You can look at the Bokeh gallery or documentation for some examples and inspiration.**

In [38]:
#Just run this cell
s = figure()

colors = ['#018b23','#407294']
xs = [covid_cali['Date'][:30]]
ys = [covid_cali['Confirmed'][:30], covid_cali['Confirmed'][30:]]
labels = ['Sept data','Oct data']

sept = s.line(x = xs[0],y = ys[0],line_color = colors[0])
october = s.line(x = xs[0],y = ys[1],line_color = colors[1])

legend = Legend(items=[
    ("Sept Data", [sept]),
    ("Oct Data" , [october]),
], location="top_right")

s.add_layout(legend, 'right')
show(s)

# Part 2: Mapping Confirmed COVID-19 Cases By States

In the second part of this lab, we are going to create an interactive map of confirmed COVID-19 cases in the U.S. by state. The darker the color is of a given state, the more cases it has. Note that since this part is more involved than the first part, we have provided a lot of the code for you, but we really encourage you to read through them and take a look at the Bokeh documentation if you are interested in what exactly each line does!

Again, let's load the table we are working with and examine its content. The way we generate the map and obtain the shape of each state is through plotting a collection of cooridnates, which is why instead of a standard csv file, we are using a shapefile that contains said geodata. And instead of reading it with pandas, we will be using GeoPandas, hence the `gpd.read_file('covid_map.shp')` line.

In [7]:
#run this cell
covid_map = gpd.read_file('covid_map.shp')
covid_map.head()

NAME  Confirmed  Deaths  Recovered    Active  People_Tes  \
0      Maryland     142425    4115     8117.0  130193.0   3333562.0   
1          Iowa     120635    1691    90532.0   28412.0    873362.0   
2      Delaware      24392     688    12934.0   10770.0    344939.0   
3          Ohio     205347    5256   163472.0   36619.0   4314037.0   
4  Pennsylvania     205852    8707   154518.0   42627.0   2469920.0   

                                            geometry  
0  MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ...  
1  POLYGON ((-96.62187 42.77925, -96.57794 42.827...  
2  POLYGON ((-75.77379 39.72220, -75.75323 39.757...  
3  MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ...  
4  POLYGON ((-80.51989 40.90666, -80.51964 40.987...

**<span style ='color:red'> After running the cell above, you should see a table with 7 columns. If you see a table with only one column named "geometry", please stop and make sure that you have imported ALL the files included in the zip file. </span>**

In [8]:
#Just run this cell
#convert our table into a geojson file that Bokeh can work with
#GeoJSON is a way of encoding geographic data (i.e. the shape of a state)
#Since Bokeh renders its graphics with JavaScript, we want to convert out file into GeoJSON data.
geosource = GeoJSONDataSource(geojson = covid_map.to_json())

### 1. Choosing palette and color range
Take a look at the [palettes](https://docs.bokeh.org/en/latest/docs/reference/palettes.html) that Bokeh have and pick one that you like to color in the map! We have provided you with an example of how to select a palette in the first line, but feel free to change that. 

In [9]:
#choose your palette here. We have chosen the palette 'BuPu' with 9 colors. 
palette = brewer['BuPu'][9][::-1] #CHANGE THIS TO ANOTHER PALETTE

Now, let's define the color range for our map. This will determine how dark and light each state should be. Fill out the first two lines to set what number the lightest color should correspond to, and what number the darkest color should correspond to. 

Hint: since we are trying to plot the number of confirmed cases, you can get the data in a given column from the `covid_map` table with this syntax: `covid_map[*column_name_here*]`

In [ ]:
#IMPORTANT: DO NOT change the variable names
min_range = #YOUR CODE HERE
max_range = #YOUR CODE HERE
color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)

Now that we have selected our palette and defined our color range, we can make a color bar that shows what color a given number roughly corresponds to.

In [25]:
#Make the ticker for the color bar 
cb_ticker = {}
for i in range(0,1000000,10000):
    cb_ticker[i] = '{:,}'.format(i)

#construct the color bar
color_bar = ColorBar(color_mapper=color_mapper, 
                     major_label_text_font_size="8pt",
                     major_label_overrides = cb_ticker,
                     label_standoff=12, 
                     border_line_color=None, 
                     location=(0, 0),
                     orientation = 'horizontal',
                     width = 1000, 
                     height = 20
                    )

Finally, let's construct our figure. First, we need to define a "blank canvas" that our map will live in. We are going to use the variable `p` to represent our graph. And we will use the `figure()` method to construct it. You should choose a descriptive title for our map.  

In [40]:
 p = figure(title = #TODO: GIVE YOUR GRAPH A TITLE HERE, 
           plot_height = 1200,
           plot_width = 5300,
           toolbar_location = 'above',
           toolbar_sticky = True)

#This makes sure there is no visible grid lines behind our map
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

Next, let's make a blank map. We will do this with the `p.patches()` method. Once we give it a source (the GeoJSON data we define a few cells above), Bokeh can extract the data from the given source and plot our map on a x-y plane. And if you hit run on the following cell, you should see a blank map of the U.S.. 

In [41]:
#Just run this cell first, read the part below this cell and fill the missing line)
states = p.patches('xs','ys', source = geosource,
                   fill_color = {#TODO:YOUR CODE HERE
                                'transform' : color_mapper},
                   line_width = 0.25, 
                   fill_alpha = 1,
                   line_color = 'grey')

show(p)

In order to fill in the map, we have to specify based on what data we want Bokeh to color the map. We can do this by adding a key called `'field'` to the `fill_color` dictionary in `p.pathces`. Since we want to color in our map based on the number of confirmed cases, the value that corresponds to the `'field'` key should be the **name of the column** from our `covid_map` table that contains our desired data. 

After you have filled in the `'field'` key, run the cell again, and you should see a colored map!

Finally, let's make our map interactive. We want to show our users more detailed information regarding COVID-19 cases in each state. For example, when they hover over a state, we want to show them the name of that state, and the confirmed number of cases in that state on Oct. 28th. To do this, we will call `p.add_tools(HoverTool())`. You can show as much or as little information you want by adding **tupules** to the list `tooltips`. The first item of the tupule correpsonds to the label of that value, and the second item should be the name of the column from the `covid_map` table. We have given you two examples on how to do this in the code below, but feel free to add more data to show!

In [42]:
#create hover tool here
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),
                                  ('Confirmed Cases','@Confirmed'),
                                 #TODO:ADD MORE TUPLES HERE. BE SURE TO FOLLOW THE FORMAT OF THE ABOVE TWO LINE
                                 ]))

#This allows us to add the color bar to the bottom of the graph
p.add_layout(color_bar, 'below')

show(p)

Congratulations! You have made an interactive map with Bokeh to visualize COVID-19 data!

### Optional/Challenge
For the sake of time and complexity, we provided you with a lot of the code for this lab. However, an optional task would be to follow the process and make a new map based on a different value from the table. For example, a colored map based on how many people were tested in each state on Oct. 28. Think about what changes you have to make (color range, tooltips, etc.) 